In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Launch Kaggle TPU session
print('Session Start!')

In [ ]:
!pip install torch_geometric
!pip install indexed_bzip2
!pip install rdkit

#Run when Accelerator is GPU P100
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU available: ', torch.cuda.get_device_name(device))
else:
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print('GPU not available, using CPU')
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch_geometric.data import Data, DataLoader
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_max_pool as gmp
import indexed_bzip2 as ibz2
import os
import pickle
import rdkit
from rdkit import Chem
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
#from torch_scatter import scatter
from multiprocessing import Pool
from tqdm import tqdm
import gc

In [ ]:
#Run when Accelerator is TPU VM v3-8
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.data_parallel as dp
import torch_xla.debug.metrics as met
import torch_xla.utils.utils as xu
import torch_xla.distributed.parallel_loader as pl

# Check TPU availability
assert xm.xrt_world_size() == 1, 'TPU not available or not properly configured'
print('TPU is available and properly configured')

# Use TPU device
device = xm.xla_device()
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch_geometric.data import Data, DataLoader
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_max_pool as gmp
import indexed_bzip2 as ibz2
import os
import pickle
import rdkit
from rdkit import Chem
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
#from torch_scatter import scatter
from multiprocessing import Pool
from tqdm import tqdm
import gc
print('import DONE!')

In [ ]:
#Load first 30m Train_graph
def load_compressed_ibz2_pickle(file):
    with ibz2.open(file, parallelization=os.cpu_count()) as f:
        data = pickle.load(f)
    return data
gdf_train = load_compressed_ibz2_pickle(
    '/kaggle/input/leash-bio-processed-dataset/train-replace-c-30m.graph.pickle.b2z'
)
print('train_graph Loaded!')

# Load Train_bind
trainbind_data = np.load('/kaggle/input/leash-bio-processed-dataset/train.bind.npz')
train_bind = trainbind_data['bind']
trainbind_data.close()
print('train_bind Loaded!')

print(len(gdf_train),len(train_bind))
print('DONE!')

In [ ]:
device

In [ ]:
#Run when loading full training dataset
#gdf_train2 = load_compressed_ibz2_pickle(
#    '/kaggle/input/leash-bio-processed-dataset/train-replace-c-30m.graph.pickle.01.b2z'
#)
#print('train_graph Loaded!')
#print(len(gdf_train2))

#gdf_train3 = load_compressed_ibz2_pickle(
#    '/kaggle/input/leash-bio-processed-dataset/train-replace-c-30m.graph.pickle.02.b2z'
#)
#print('train_graph Loaded!')
#print(len(gdf_train3))

In [ ]:
#merge train_graph datasets
gdf_train_concat =  gdf_train #+ gdf_train2 + gdf_train3  #add train2 and train3 when loading full training dataset
print("DONE",len(gdf_train_concat))

In [ ]:
zipped = list(zip(gdf_train_concat[:10000000],train_bind[:10000000]))

pos_class = [x for x in zipped if sum(x[1]) > 0]
neg_class = [x for x in zipped if sum(x[1]) == 0]
print(len(pos_class),len(neg_class))
print("DONE!")

In [ ]:
#skip when training for submission
import random
random.shuffle(pos_class)
random.shuffle(neg_class)

train_sample = pos_class[:100000] + neg_class[:100000]
train_graph_sample = []
train_bind_sample = []

random.shuffle(train_sample)

for i in train_sample:
    train_graph_sample.append(i[0])
    train_bind_sample.append(i[1])
    
    
print('length of train split: ',len(train_graph_sample), len(train_bind_sample))


In [ ]:
#check train_bind 
train_bind_sample[:5]

In [ ]:
#Helper: convert graph to pyg list
def to_pyg_list(graph):
    L = len(graph)
    for i in tqdm(range(L)):
        N, edge, node_feature, edge_feature = graph[i]
        graph[i] = Data(
            idx=i,
            edge_index=torch.from_numpy(edge.T).int(),
            x=torch.from_numpy(node_feature).byte(),
            edge_attr=torch.from_numpy(edge_feature).byte(),
            y=torch.tensor(train_bind_sample[i])
        )
    return graph

#Helper for test_split
def to_pyg_list_test(graph):
    L = len(graph)
    for i in tqdm(range(L)):
        N, edge, node_feature, edge_feature = graph[i]
        graph[i] = Data(
            idx=i,
            edge_index=torch.from_numpy(edge.T).int(),
            x=torch.from_numpy(node_feature).byte(),
            edge_attr=torch.from_numpy(edge_feature).byte()
        )
    return graph

# Converted test_graph, with label
def to_pyg_list_tests_wlabel(graph):
    L = len(graph)
    for i in tqdm(range(L)):
        N, edge, node_feature, edge_feature = graph[i]
        graph[i] = Data(
            idx=i,
            edge_index=torch.from_numpy(edge.T).int(),
            x=torch.from_numpy(node_feature).byte(),
            edge_attr=torch.from_numpy(edge_feature).byte(),
            y=torch.tensor(test_bind_sample[i])
        )
    return graph
print('DONE!')

In [ ]:
# Converted train-graph, with label
train_graph = to_pyg_list(train_graph_sample)
#test_graph_wlabel = to_pyg_list_tests_wlabel(test_graph_sample)
print(len(train_graph))
train_graph[:5]

In [ ]:
class GATNet(torch.nn.Module):
    def __init__(self, num_features=9, n_output=3, output_dim=32, dropout=0.5):
        super(GATNet, self).__init__()

        self.gcn1 = GATConv(num_features, num_features, heads=10, dropout=dropout)
        self.gcn2 = GATConv(num_features * 10, output_dim, dropout=dropout)
        self.fc_g1 = nn.Linear(output_dim, output_dim)

        # add batch normalization
        self.bn1 = nn.BatchNorm1d(output_dim)

        self.fc1 = nn.Linear(output_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.out = nn.Linear(32, n_output)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x, edge_index, batch = data.x.float(), data.edge_index, data.batch

        x = F.dropout(x, p=0.2, training=self.training)
        x = F.elu(self.gcn1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.gcn2(x, edge_index)
        x = self.bn1(x)
        x = self.relu(x)
        x = gmp(x, batch)          
        x = self.fc_g1(x)
        x = self.relu(x)

        # Dense layers
        xc = self.fc1(x)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out

model = GATNet()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

def custom_loss(output, target):
    output = output.view(-1)
    target = target.view(-1)
    return F.binary_cross_entropy_with_logits(output, target.float())

def train_model(training_set, batch_size=100000, epochs=8):
    model.train()
    dataloader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    for epoch in range(epochs):
        total_loss = 0
        for data in dataloader:
            optimizer.zero_grad()
            output = model(data)
            loss = custom_loss(output, data.y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            optimizer.step()
            total_loss += loss.item()
            
        # update learning rate
        scheduler.step()  
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')

model = model.to(device)
def train_model(training_set, batch_size=50000, epochs=8):
    model.train()
    dataloader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    for epoch in range(epochs):
        total_loss = 0
        for data in dataloader:
            data = data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = custom_loss(output, data.y)
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
            xm.optimizer_step(optimizer)
            total_loss += loss.item()
            
        # update learning rate
        xm.mark_step()  
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')

In [ ]:
# For Debug
training_set = train_graph#[:10000]#[Data(idx=-1, edge_index=None, x=torch.rand(5, 20), edge_attr=None, y=torch.tensor([0, 0, 1])) for _ in range(10)]

# Train with the training_set
train_model(training_set)
print('training DONE!')

In [ ]:
# Prediction function
def predict_label(data):
    model.eval()
    output = model(data)
    labels = torch.sigmoid(output).detach().numpy()
    #predicted_label = 0 if np.mean(labels)<0.1 else 1
    return labels#predicted_label

In [ ]:
#Threshold
test_sample = pos_class[100000:110000] + neg_class[100000:110000]
test_graph_sample = []
test_bind_sample = []

random.shuffle(train_sample)

for i in test_sample:
    test_graph_sample.append(i[0])
    test_bind_sample.append(i[1])
    
print('length of test split: ',len(test_graph_sample), len(test_bind_sample))

#Convert testset to graph object
def to_pyg_list_test(graph):
    L = len(graph)
    for i in tqdm(range(L)):
        N, edge, node_feature, edge_feature = graph[i]
        graph[i] = Data(
            idx=i,
            edge_index=torch.from_numpy(edge.T).int(),
            x=torch.from_numpy(node_feature).byte(),
            edge_attr=torch.from_numpy(edge_feature).byte(),
            y=torch.tensor(test_bind_sample[i])
        )
    return graph

#test_sample = pos_class[-5:]+neg_class[-5:]
test_graph_converted = to_pyg_list_test(test_graph_sample)

print('DONE!')
test_graph_converted[:5]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, global_mean_pool as gmp
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

def evaluate_model(test_set):
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for data in test_set:
            output = model(data)
            output_sigmoid = output.sigmoid().numpy()
            binary_labels = (np.sum(data.y.numpy()) > 0).astype(int)
            all_labels.extend([binary_labels])
            all_preds.extend(output_sigmoid.max(axis=1))  # Using max of three class, assuming positive correlation
    
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    
    #fpr, tpr, _ = roc_curve(all_labels, all_preds)
    #roc_auc = auc(fpr, tpr)
    
    #plt.figure()
    #plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    #plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.05])
    #plt.xlabel('False Positive Rate')
    #plt.ylabel('True Positive Rate')
    #plt.title('Receiver Operating Characteristic')
    #plt.legend(loc="lower right")
    #plt.show()
    return all_labels, all_preds

lanels_, preds_ = evaluate_model(test_graph_converted)

In [ ]:
#check output in preds_
preds_[:5]

In [ ]:
threshold = np.round(preds_[0],4) - 0.03
stepsize = 0.0001
match_lst = []
threshold_lst = []
high_thr, high_match = -1, -1
low_thr, low_match = 1, 1
counter, last_match = 0, 0
while threshold <=np.round(preds_[0],4) + 0.03:
    binary_pred_ = [1 if x <= threshold else 0 for x in preds_]
    match = np.round(np.mean([1 if x==y else 0 for x,y in zip(binary_pred_, lanels_)]),4)

        
    if match > high_match: #update best-accuracy threshold
        high_match = match
        high_thr = np.round(threshold,5)
        
    elif match < low_match: #early stop if accuracy is dropping continuously
        low_match = match
        low_thr = np.round(threshold,5)
        
    match_lst.append(match)
    threshold_lst.append(np.round(threshold,4))
    threshold += stepsize
print('DONE')

plt.figure()
plt.plot(threshold_lst, match_lst, color='darkorange', lw=2, label='acc')
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
print(high_match, high_thr, low_match, low_thr)
match_diff_h, match_diff_l = abs(high_match)-0.5, abs(low_match-0.5)
print(match_diff_h, match_diff_l)
if high_match == -1 or match_diff_h < match_diff_l:
    rev = 1

else:
    rev = 0
    
rev


In [ ]:
# Prediction test
import random
for i in range(10):
    print(predict_label(test_graph_converted[i]))

In [ ]:
#Convert SMILES to graph objects
PACK_NODE_DIM=9
PACK_EDGE_DIM=1
NODE_DIM=PACK_NODE_DIM*8
EDGE_DIM=PACK_EDGE_DIM*8

def one_of_k_encoding(x, allowable_set, allow_unk=False):
    if x not in allowable_set:
        if allow_unk:
            x = allowable_set[-1]
        else:
            raise Exception(f'input {x} not in allowable set{allowable_set}!!!')
    return list(map(lambda s: x == s, allowable_set))


#Get features of an atom (one-hot encoding:)
'''
    1.atom element: 44+1 dimensions    
    2.the atom's hybridization: 5 dimensions
    3.degree of atom: 6 dimensions                        
    4.total number of H bound to atom: 6 dimensions
    5.number of implicit H bound to atom: 6 dimensions    
    6.whether the atom is on ring: 1 dimension
    7.whether the atom is aromatic: 1 dimension           
    Total: 70 dimensions
'''

ATOM_SYMBOL = [
    'C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg',
    'Na', 'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl',
    'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H',
    'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
    'Pt', 'Hg', 'Pb', 'Dy',
    #'Unknown'
]
#print('ATOM_SYMBOL', len(ATOM_SYMBOL))44
HYBRIDIZATION_TYPE = [
    Chem.rdchem.HybridizationType.S,
    Chem.rdchem.HybridizationType.SP,
    Chem.rdchem.HybridizationType.SP2,
    Chem.rdchem.HybridizationType.SP3,
    Chem.rdchem.HybridizationType.SP3D
]

def get_atom_feature(atom):
    feature = (
        one_of_k_encoding(atom.GetSymbol(), ATOM_SYMBOL)
       + one_of_k_encoding(atom.GetHybridization(), HYBRIDIZATION_TYPE)
       + one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5])
       + one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5])
       + one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5])
       + [atom.IsInRing()]
       + [atom.GetIsAromatic()]
    )
    #feature = np.array(feature, dtype=np.uint8)
    feature = np.packbits(feature)
    return feature


#Get features of an edge (one-hot encoding)
'''
    1.single/double/triple/aromatic: 4 dimensions       
    2.the atom's hybridization: 1 dimensions
    3.whether the bond is on ring: 1 dimension          
    Total: 6 dimensions
'''

def get_bond_feature(bond):
    bond_type = bond.GetBondType()
    feature = [
        bond_type == Chem.rdchem.BondType.SINGLE,
        bond_type == Chem.rdchem.BondType.DOUBLE,
        bond_type == Chem.rdchem.BondType.TRIPLE,
        bond_type == Chem.rdchem.BondType.AROMATIC,
        bond.GetIsConjugated(),
        bond.IsInRing()
    ]
    #feature = np.array(feature, dtype=np.uint8)
    feature = np.packbits(feature)
    return feature


def smile_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    N = mol.GetNumAtoms()
    node_feature = []
    edge_feature = []
    edge = []
    for i in range(mol.GetNumAtoms()):
        atom_i = mol.GetAtomWithIdx(i)
        atom_i_features = get_atom_feature(atom_i)
        node_feature.append(atom_i_features)

        for j in range(mol.GetNumAtoms()):
            bond_ij = mol.GetBondBetweenAtoms(i, j)
            if bond_ij is not None:
                edge.append([i, j])
                bond_features_ij = get_bond_feature(bond_ij)
                edge_feature.append(bond_features_ij)
    node_feature=np.stack(node_feature)
    edge_feature=np.stack(edge_feature)
    edge = np.array(edge,dtype=np.uint8)
    return N,edge,node_feature,edge_feature

def to_pyg_format(N,edge,node_feature,edge_feature):
    graph = Data(
        idx=-1,
        edge_index = torch.from_numpy(edge.T).int(),
        x          = torch.from_numpy(node_feature).byte(),
        edge_attr  = torch.from_numpy(edge_feature).byte(),
    )
    return graph

#debug one example
g = to_pyg_format(*smile_to_graph(smiles="C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1"))
print(g)
print('[Dy] is replaced by C !!')
print('smile_to_graph() ok!')

In [ ]:
# Prediction function
def predict_label(data):
    model.eval()
    output = model(data)
    labels = torch.sigmoid(output).detach().numpy()
    #predicted_label = 0 if np.mean(labels)<0.1 else 1
    return labels#predicted_label

test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission_GAT.csv'  # Specify the path and filename for the output file
count = 0
# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=10000):
    def helper_(s):
        return to_pyg_format(*smile_to_graph(smiles=s))
    #df_test.drop(["buildingblock1_smiles","buildingblock2_smiles","buildingblock3_smiles"],axis = 1)
    df_test_graph = df_test['molecule_smiles'].apply(helper_)
    

    # 
    pred = df_test_graph.apply(predict_label)
    
    # Predict the probabilities
    def helper2(pred_ar):
        if rev == 0:
            if pred_ar.max(axis=1)[0] <= high_thr:
                return 1
            return 0
        else:
            if pred_ar.max(axis=1)[0] <= low_thr:
                return 0
            return 1
        
        
    probabilities = pred.apply(helper2)

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
    count += 1
    print('iteration completed: ',count)
print('Prediction DONE! Result saved to ',output_file)